## 4-4　Transformers

In [1]:
!pip install transformers==4.5.0
!pip install fugashi==1.1.0
!pip install ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 596 kB/s            
     |████████████████████████████████| 3.3 MB 64.4 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
     |████████████████████████████████| 486 kB 597 kB/s            
     |████████████████████████████████| 13.4 MB 526 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=0441d160327598fdba10ba6bf97bcf34cbce28d90cd8133ead00c97a716f28d2
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [2]:
import torch
from transformers import BertJapaneseTokenizer, BertModel

### 4-4-1　トークナイザ

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

In [4]:
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [5]:
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [6]:
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

トークンを符号化する。

In [7]:
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう。')
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


ID をトークン化する。

In [8]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

In [9]:
text = '明日の天気は晴れだ。'
encoding = tokenizer(text, max_length=12, padding='max_length', truncation=True)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


`token_type_ids` は、2つの文章のペアを区別するときに使用する。

In [10]:
encoding = tokenizer(text, max_length=6, padding='max_length', truncation=True)
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [11]:
text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']
tokenizer(text_list, max_length=10, padding='max_length', truncation=True)

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [12]:
tokenizer(text_list, padding='longest')

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [13]:
tokenizer(text_list, max_length=10, padding='max_length', truncation=True, return_tensors='pt')

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 4-4-2　BERT モデル

In [14]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

bert = bert.cuda()

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [15]:
print(bert.config)

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [16]:
text_list = [
    '明日は自然言語処理の勉強をしよう',
    '明日はマシーンラーニングの勉強をしよう'
]

encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoding = {k: v.cuda() for k, v in encoding.items()}

output = bert(**encoding)
last_hidden_state = output.last_hidden_state

In [17]:
print(last_hidden_state.size())

torch.Size([2, 32, 768])


In [18]:
with torch.no_grad():
    output = bert(**encoding)
    last_hidden_state = output.last_hidden_state

In [19]:
last_hidden_state = last_hidden_state.cpu()
last_hidden_state = last_hidden_state.numpy()
last_hidden_state = last_hidden_state.tolist()